In [ ]:
# regex para excluir caracteres especiais (eg., '|') -- DONE
# testar spacy e o outro multilingual large
# tecnicas para reduzir quantitativo no -1
# tecnicas para reduzir a quantidade de tópicos

In [1]:
DATA_PATH = "P:/study/master/covid-news-brazil/raw"

import os
import re
import spacy

import pandas as pd
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

def preprocess(text):
    text = text.replace('\n', ' ')
    text = text.replace(u'\xa0', u' ')
    text = re.sub(r'[^ \w+]', '', text)
    text = ' '.join(text.split())
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return (" ".join(new_text)).strip()

In [85]:
nlp = spacy.load("pt_core_news_md")
data = []
for doc in nlp.pipe(docs[:10], disable=["tok2vec", "tagger", "parser", "attribute_ruler"]):
    data.append([
        re.sub(r'[^\w\s]|[0-9]|[_]', '', token.lower()) for token in doc
        if not token.like_url
        and not token.is_punct
        and not token.is_stop
        and not token.is_space
        and not token.text.startswith('@')
    ])

OSError: [E050] Can't find model 'pt_core_news_md'. It doesn't seem to be a Python package or a valid path to a data directory.

In [10]:
news = pd.read_csv(os.path.join(DATA_PATH, "news_tweets.csv"))
accounts = pd.read_csv(os.path.join(DATA_PATH, "news_accounts.csv"))
news['created_at'] = pd.to_datetime(news.created_at, utc=True)
news = news.merge(accounts[['author_id', 'display_name']])

In [16]:
accounts.merge(news[['author_id']])[['author_id', "display_name"]].drop_duplicates()

,author_id,display_name
0,24802340,Jornal Extra
11445,790680,BBC News Brasil
21728,809124436880322561,CNN Brasil
45099,65473559,Portal R7.com
62445,14594698,UOL Notícias
89655,14594813,Folha de S.Paulo
110173,8802752,g1
124780,9317502,Estadão 🗞️
141756,54341363,Jornal O Globo
159678,17715048,VEJA


In [20]:
top = news[news.display_name.isin([
    "UOL Notícias", "Folha de S.Paulo", "g1", "Jornal O Globo", "Portal R7.com"
])].reset_index()
docs = list(top.text.apply(lambda x: preprocess(x)))
docs = [d for d in docs if len(d.split())>3]

## V6

In [26]:
from scipy.cluster import hierarchy as sch

In [23]:
sentence_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
topic_model = BERTopic(embedding_model=sentence_model, min_topic_size=50, calculate_probabilities=False)
topics, probs = topic_model.fit_transform(docs)

In [24]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,36033,-1_covid19_pandemia_de_covid
1,0,18622,0_coronavírus_do_novo_por
2,1,3922,1_morre_internado_covid19_anos
3,2,2406,2_lockdown_reabertura_bares_aglomeração
4,3,1673,3_coronavac_butantan_doses_anvisa
...,...,...,...
139,138,53,138_papa_francisco_igreja_fiéis
140,139,53,139_dose_população_tomaram_sp
141,140,52,140_hiv_prioritário_inclui_aids
142,141,51,141_milhões_doses_vacinas_ministério


In [27]:
# Hierarchical topics
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = topic_model.hierarchical_topics(docs, topics, linkage_function=linkage_function)

AttributeError: 'BERTopic' object has no attribute 'hierarchical_topics'

In [25]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

NameError: name 'hierarchical_topics' is not defined

## V5

In [91]:
sentence_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
topic_model = BERTopic(embedding_model=sentence_model, nr_topics=6, min_topic_size=25, calculate_probabilities=False)
topics, probs = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

In [93]:
topics, probs = topic_model.fit_transform(docs)

In [97]:
folha['n'] = folha.text.apply(lambda x: len(x.split())>3)

In [102]:
folha = folha[folha.n]

In [112]:
d = pd.DataFrame(docs, columns=['text'])

In [113]:
d['topic'] = topics

In [116]:
list(d[d.topic==0].text)

['Anvisa recebe primeiro pedido de registro do autoteste para Covid',
 'Subvariante da ômicron é mais infecciosa do que original diz estudo',
 'Farmácias que venderam autoteste de Covid sem registro retiram produto da prateleira',
 'Na volta às aulas alunos trocam relato minhas férias por minha vacina',
 'Cantor do BTS Jimin é internado por apendicite e recebe diagnóstico de Covid',
 'PF conclui que Bolsonaro não cometeu crime de prevaricação no caso da compra de vacina',
 'Kanye West não poderá fazer show na Austrália sem vacina diz governo',
 'Passaporte vacinal contra Covid atacado por Bolsonaro é comum em outros países',
 'Leitores comentam obrigatoriedade de comprovante de vacina',
 'Com Covid19 Maju Coutinho não apresenta o Fantástico deste domingo',
 'Adolescência em tempos de Covid como jovens estão enfrentando o prolongamento da pandemia',
 'Oficiais do Exército na Saúde comandaram contratação e distribuição falha de vacina infantil',
 'Governo de SP determina que alunos mostr

In [94]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,11347,0_coronavírus_de_do_em
1,-1,7134,-1_de_pandemia_da_na
2,1,391,1_cloroquina_hidroxicloroquina_uso_tratamento
3,2,238,2_coronavac_butantan_doses_doria
4,3,136,3_máscara_uso_máscaras_usar
5,4,121,4_auxílio_emergencial_600_prorrogação
6,5,112,5_escolas_aulas_às_volta
7,6,98,6_manaus_oxigênio_em_amazonas
8,7,91,7_pandemia_bilhões_bi_governo
9,8,73,8_leitor_empreendedor_prêmio_premiação


## V4

In [87]:
umap_model = UMAP(n_neighbors=20, n_components=10, metric='cosine', low_memory=False)
hdbscan_model = HDBSCAN(
    min_cluster_size=10, metric='euclidean',
    cluster_selection_method='eom', prediction_data=True, min_samples=5
)

topic_model = BERTopic(
    language="multilingual", umap_model=umap_model, hdbscan_model=hdbscan_model,
    min_topic_size=25, nr_topics="auto", calculate_probabilities=False
)
topics, probs = topic_model.fit_transform(docs)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.1.2 requires Cython==0.29.23, but you have cython 0.29.30 which is incompatible.


     --------------------------------------- 51.5/51.5 MB 42.4 kB/s eta 0:00:00
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstalled smart-open-6.0.0
[+] Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [88]:
topic_model.get_topic_info()

,Topic,Count,Name
0,0,10263,0_de_da_em_para
1,-1,9271,-1_de_da_em_que
2,1,285,1_china_chinês_coronavírus_chinesa
3,2,80,2_lockdown_decreta_de_em
4,3,65,3_voos_aéreas_coronavírus_suspende
5,4,60,4_oxigênio_manaus_falta_amazonas
6,5,57,5_amazonas_amazônia_amazon_do
7,6,53,6_itália_coronavírus_mortes_quarentena
8,7,50,7_passaporte_vacina_da_frias
9,8,47,8_cinemas_cinema_hollywood_teatros


## V3

In [71]:
umap_model = UMAP(n_neighbors=25, n_components=10, metric='cosine', low_memory=False)

topic_model = BERTopic(language="multilingual", umap_model=umap_model, n_gram_range=(1,3), min_topic_size=25, nr_topics="auto", calculate_probabilities=False)
topics, probs = topic_model.fit_transform(docs)

In [72]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,9689,-1_de_da_em_do
1,0,9257,0_de_da_em_que
2,1,347,1_máscara_de máscara_de_máscaras
3,2,213,2_china_coronavírus_de_chinês
4,3,120,3_cloroquina_de cloroquina_uso_uso de cloroquina
5,4,93,4_lockdown_lockdown em_de_em
6,5,88,5_ômicron_variante_variante ômicron_da variante
7,6,59,6_trump_de trump_que_de
8,7,58,7_itália_coronavírus_itália tem_coronavírus it...
9,8,57,8_podcast_ouça_ouça podcast_da


## Updated Preprocessing

In [68]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,7760,-1_coronavírus_pandemia_de_do
1,0,434,0_contra_vacina_dose_vacinas
2,1,412,1_bolsonaro_vacina_vacinas_aids
3,2,366,2_cpi_senadores_comissão_renan
4,3,334,3_brasil_vacinas_doses_vacina
...,...,...,...
251,250,10,250_disseminado_adotadas_contenção_tinha
252,251,10,251_gustavo_ator_retrospectiva2021_fernão
253,252,10,252_museu_língua_portuguesa_louvre
254,253,10,253_unam_power_soft_acelerando


## RAW

In [9]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,7437,-1_de_da_pandemia_coronavírus
1,0,1354,0_bolsonaro_presidente_cpi_jair
2,1,688,1_contra_vacina_dose_vacinação
3,2,411,2_bolsonaro_vacina_aids_vacinas
4,3,338,3_brasil_brasileiros_coronavírus_variante
...,...,...,...
237,236,10,236_crianças_adolescentes_injustificável_tétano
238,237,10,237_torna_acostumaram_desumanidade_atualizado
239,238,10,238_vendedor_propina_negociação_wizard
240,239,10,239_imobiliário_mercado_agitam_57
